In [1]:
import os

os.environ["SEED"] = "0"

import json
from pathlib import Path

import cocoex
import pandas as pd
import numpy as np

from src.constant import DATABASE_DIR
from src.database import DB
from src.instance.BBOB_Instance import BBOB_Instance
from src.instance.InstanceList import InstanceList
from src.solver.BBOB_POP_Solver import BBOB_POP_Solver
from src.solver.Portfolio import Portfolio

pd.options.display.max_columns = None

In [2]:
features = {}

for features_file in Path("features").glob("*.json"):
    with open(features_file, "r") as f:
        features = {
            **features,
            **json.load(f)
        }

len(features)

200

In [3]:
_ = pd.DataFrame([v["result"] for v in features.values()])
print(_.isna().sum().sum())
print(_.max().max())
print(_.min().min())

0
36412799.976986
-296.503722


In [4]:
# from src.instance.BBOB_Instance import BBOB_Instance
# import json

# features = {}

# for instance_index in [1, 2, 3, 4, 5]:
#     instance = BBOB_Instance(
#         function_index=7,
#         dimension=2,
#         instance_index=instance_index,
#     )
#     result, time = instance.calculate_features()
#     features[instance.id()] = {"result": result, "time": time}

# with open("features/features_7_2.json", "w") as f:
#     json.dump(features, f, indent=4)

In [75]:
db = DB(DATABASE_DIR / "run-phase1-dataset-baseline--25-1128432.db")

In [59]:
instances_df = db.get_instances()
solvers_df = db.get_solvers()
evaluations_df = db.get_evaluations()

df = pd.merge(evaluations_df.drop(columns="id"), solvers_df, left_on="solver_id", right_on="id").drop(columns=["id"])
df = pd.merge(df, instances_df, left_on="instance_id", right_on="id").drop(columns=["id"])
df

,solver_id,instance_id,cost,ALGORITHM,CMA_ELITIST,CMA_POPSIZE,CMA_POPSIZE_FACTOR,CMA_RANDOM_INIT,CMA_SCALE,DE_CROSSOVER,DE_F1,DE_F2,DE_INITIALIZATION,DE_POPSIZE,DE_RECOMMENDATION,DE_SCALE,PSO_OMEGA,PSO_PHIG,PSO_PHIP,PSO_POPSIZE,function_index,dimension,instance_index
0,410950163550714701,122021300402733609,0.486759,0.0,1.0,0.955556,0.410492,1.0,0.972919,0.081101,0.725594,0.089603,2.0,0.311111,1.0,0.695625,0.056848,0.373813,0.459856,0.922222,2,2,1
1,1743092914995070369,122021300402733609,0.523542,0.0,1.0,0.600000,0.246063,1.0,0.447144,0.449083,0.897070,0.891523,1.0,0.711111,3.0,0.351871,0.519643,0.707917,0.696077,0.111111,2,2,1
2,2283111023303066572,122021300402733609,0.084861,1.0,0.0,0.244444,0.426904,1.0,0.406922,0.772266,0.224467,0.705148,0.0,0.066667,3.0,0.526907,0.208253,0.159146,0.811939,0.044444,2,2,1
3,2253826169615980878,122021300402733609,0.793972,1.0,1.0,0.111111,0.118728,1.0,0.916723,0.842342,0.552821,0.703737,2.0,0.677778,0.0,0.675439,0.659668,0.474868,0.749170,0.011111,2,2,1
4,142531513711967916,122021300402733609,1.050796,2.0,0.0,0.088889,0.311860,1.0,0.730856,0.207470,0.586784,0.716860,2.0,0.500000,0.0,0.961936,0.929529,0.063955,0.961570,0.544444,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,708619547354767275,285578388822138370,3000.000000,0.0,0.0,0.233333,0.384026,0.0,0.714490,0.829189,0.304510,0.559249,0.0,0.433333,0.0,0.304533,0.994398,0.969112,0.555636,0.200000,2,20,1
4996,1381777441855865549,285578388822138370,3000.000000,1.0,0.0,0.866667,0.797349,1.0,0.611219,0.673328,0.043712,0.317373,0.0,0.700000,0.0,0.160687,0.734556,0.765746,0.112657,0.633333,2,20,1
4997,410443325615465788,285578388822138370,4.109964,1.0,0.0,0.911111,0.450473,0.0,0.908022,0.574829,0.328057,0.342551,2.0,0.400000,3.0,0.747411,0.411143,0.670383,0.782931,0.522222,2,20,1
4998,396791986407096231,285578388822138370,3000.000000,0.0,1.0,1.000000,0.586070,0.0,0.544160,0.658862,0.634687,0.790705,3.0,0.066667,0.0,0.004469,0.999746,0.443400,0.409085,0.777778,2,20,1


In [63]:
df.loc[df["dimension"] == 2].loc[lambda x: x["cost"] < 0.7].shape

(408, 23)

In [66]:
df.loc[df["dimension"] == 20].loc[lambda x: x["cost"] < 20.0]

,solver_id,instance_id,cost,ALGORITHM,CMA_ELITIST,CMA_POPSIZE,CMA_POPSIZE_FACTOR,CMA_RANDOM_INIT,CMA_SCALE,DE_CROSSOVER,DE_F1,DE_F2,DE_INITIALIZATION,DE_POPSIZE,DE_RECOMMENDATION,DE_SCALE,PSO_OMEGA,PSO_PHIG,PSO_PHIP,PSO_POPSIZE,function_index,dimension,instance_index
4001,1743092914995070369,285578388822138370,3.409858,0.0,1.0,0.600000,0.246063,1.0,0.447144,0.449083,0.897070,0.891523,1.0,0.711111,3.0,0.351871,0.519643,0.707917,0.696077,0.111111,2,20,1
4004,142531513711967916,285578388822138370,8.220031,2.0,0.0,0.088889,0.311860,1.0,0.730856,0.207470,0.586784,0.716860,2.0,0.500000,0.0,0.961936,0.929529,0.063955,0.961570,0.544444,2,20,1
4005,1789178219950618652,285578388822138370,9.399281,2.0,0.0,0.355556,0.998847,1.0,0.411397,0.420075,0.764912,0.338985,0.0,0.388889,3.0,0.589910,0.186193,0.254356,0.377752,0.811111,2,20,1
4008,275871620798617955,285578388822138370,11.494974,2.0,1.0,0.677778,0.042976,1.0,0.886305,0.440264,0.059944,0.672582,3.0,0.766667,2.0,0.284000,0.337477,0.591027,0.721407,0.700000,2,20,1
4010,2263293195967791625,285578388822138370,12.891729,2.0,0.0,0.800000,0.277774,0.0,0.253643,0.680514,0.676242,0.710337,4.0,0.588889,0.0,0.695270,0.600392,0.813679,0.442036,0.055556,2,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,1931658275364094990,285578388822138370,9.088923,2.0,1.0,0.033333,0.789012,0.0,0.452189,0.199584,0.447570,0.808077,2.0,0.144444,3.0,0.720836,0.356203,0.607546,0.990520,0.955556,2,20,1
4988,1983416043296189767,285578388822138370,3.514965,0.0,1.0,0.333333,0.868146,1.0,0.320785,0.522228,0.672947,0.436959,2.0,0.433333,2.0,0.946210,0.451891,0.752451,0.299583,0.288889,2,20,1
4989,792218999111593700,285578388822138370,13.398867,0.0,1.0,0.500000,0.556334,0.0,0.097171,0.194046,0.477945,0.141691,3.0,0.422222,3.0,0.031815,0.603397,0.796412,0.751455,0.077778,2,20,1
4991,2254049317544309715,285578388822138370,15.019130,0.0,1.0,0.188889,0.827818,1.0,0.847134,0.139498,0.504214,0.225796,1.0,0.011111,0.0,0.545739,0.825397,0.138665,0.561738,1.000000,2,20,1


In [72]:
(df["cost"] == 3000.0).mean()

0.2678

In [73]:
df.groupby("dimension").apply(lambda x: (x["cost"] == 3000.0).mean())

dimension
2     0.106
3     0.149
5     0.227
10    0.386
20    0.471
dtype: float64